# Librerias

In [1]:
import pandas as pd 
import string
from nltk.tokenize import word_tokenize 
from string import punctuation 
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import numpy as np

# Archivos

In [2]:
with open('archivos/negacion.txt') as archivo_negaciones:
    listaNegaciones = [line.rstrip('\n') for line in archivo_negaciones]

with open('archivos/stopwords.txt') as archivo_stopword:
    listaStopword=[line.rstrip('\n') for line in archivo_stopword]

with open('archivos/intensificadores.txt') as archivo_intensificadores:
    listaInten=[line.rstrip('\n') for line in archivo_intensificadores]
    
with open('archivos/mitigadores.txt') as archivo_mitigadores:
    listaMiti= [line.rstrip('\n') for line in archivo_mitigadores]
    
df = pd.read_excel('archivos/lexicon-SENTIMIENTO.xlsx', sheet_name='Hoja1')
listaPalabras = df['Palabras']
listaPolaridad = df['Polaridad']

# Funciones

## Filtrar StopWord

In [3]:
def Filtrar_StopWord(frase):
    frase = re.sub('[%s]' % re.escape(string.punctuation), ' ', frase)
    vector_frase = word_tokenize(frase)
    fraseFiltrada =""
    non_words = list(punctuation) 
    non_words.extend(['¿', '¡'])
    for palabra in vector_frase:
        if palabra.lower() not in lista_Stopword and palabra not in non_words:
            fraseFiltrada +=" "+palabra
    return fraseFiltrada

## Tokenize

In [4]:
def Tokenize(text):
        tokens = nltk.word_tokenize(text,  language='spanish')
        stems = []
        for item in tokens:
            stems.append(nltk.PorterStemmer().stem(item))
        return stems

## Limpiar Corpus

In [5]:
def LimpiarCorpus(X_frases):
        for i in range(len(X_frases)):
            X_frases[i] = re.sub("[^a-zA-Z]", " ", X_frases[i].lower())
            X_frases[i] = ' '.join(X_frases[i].split())
        return X_frases

## Entrenar y medir modelos ML

In [6]:
def EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo):
    # MODELO
    if(algoritmo=="SVM"):
        modelo = svm.SVC(kernel='linear') 
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Random Forest"):
        modelo=RandomForestClassifier(n_estimators=10)
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Naive Bayes"):
        modelo = GaussianNB()
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    # METRICAS
    #reporteMetricas= classification_report(Y_test, prediccion)
    return metrics.accuracy_score(Y_test, prediccion),modelo

## Buscar la palabra en el lexicón

In [7]:
def BuscarLexicon(palabra):
    for i in range (0,len(listaPalabras)):
      if palabra ==str(listaPalabras[i]):
          polaridadPalabra=str(listaPolaridad[i])
          if polaridadPalabra=='positivo':
              return 1
          else:
              if polaridadPalabra=='negativo':
                 return -1
              else:
                  return 0
    return -5

## Buscar cuantificadores

In [8]:
def BuscarCuantificadores(palabra):
    if palabra in listaInten:
        return 0.3
    else:
        if palabra in listaMiti:
            return -0.3
        return 0

## Tratar los cuantificadores

In [9]:
def TratarCuantificadores(cuantificadores,tope):
    if tope!=0:
        if cuantificadores[tope-1]>0:
            return tope*0.3*1
        else:
            return tope*-0.3*1
    else:
        return 0

## Calcular la polaridad con lexicón de una frase

In [10]:
def CalcularPolaridad(fraseFiltrada):
    numeroPositivas = 0
    numeroNegativas = 0
    polaridad=0
    negacion=0
    cuantificadores=[]
    tope=0
    for palabra in fraseFiltrada:
        if palabra in listaNegaciones:
            negacion=3
        else:
            porcentaje=BuscarCuantificadores(palabra)
            if porcentaje != 0:
                    cuantificadores.insert(tope,porcentaje)
                    tope=tope+1
            else:
                polaridadPalabra=BuscarLexicon(palabra)
                if polaridadPalabra != -5:
                    if polaridadPalabra==1: numeroPositivas +=1
                    else: numeroNegativas =+1
                    multi=TratarCuantificadores(cuantificadores,tope)
                    tope=0
                    if(negacion>0):
                        polaridad=polaridad+polaridadPalabra*-1+multi
                        negacion=negacion-1
                    else:
                        if polaridadPalabra >0:
                            polaridad=polaridad+polaridadPalabra+multi
                        else: 
                            polaridad=polaridad+polaridadPalabra-multi
    return numeroPositivas, numeroNegativas, len(cuantificadores),polaridad

## Filtrar frases

In [11]:
def FiltrarFrase (frase):
    vector_frase = word_tokenize(frase.lower())
    fraseFiltrada = []
    non_words = list(punctuation) 
    non_words.extend(['¿', '¡'])
    for palabra in vector_frase:
        if palabra not in listaStopword and palabra not in non_words:
            fraseFiltrada.append(palabra)
        else:
            if palabra in listaNegaciones or palabra in listaInten or palabra in listaMiti:
                fraseFiltrada.append(palabra)
    return fraseFiltrada

## Enfoque 2 con vector derivado de un analisis de sentimiento con lexicones

In [12]:
def ClasificarPolaridadLexicon(corpus,algoritmo):
    
    X_frases = np.asarray(corpus[['A']])
    X_frases = X_frases.ravel()
    Y_sentimiento = np.asarray(corpus[['B']])
    Y_sentimiento = Y_sentimiento.ravel()
    X_frases = LimpiarCorpus(X_frases)
    # PARTIR CORPUS PARA ENTRENAR
    X_trainPre, X_testPre, Y_train, Y_test = train_test_split(X_frases, Y_sentimiento, test_size=0.30)
    X_train = []
    X_test = []
    for i in range (0,len(X_trainPre)):
        fraseProcesada = FiltrarFrase(X_trainPre[i])
        numeroPositivas, numeroNegativas,numeroCuantificadores,polaridadFrase = CalcularPolaridad(fraseProcesada)
        X_train.insert(i,[polaridadFrase,numeroPositivas,numeroNegativas,numeroCuantificadores,])
        if(i<len(X_testPre)):
            fraseProcesada = FiltrarFrase(X_testPre[i])
            numeroPositivas, numeroNegativas, numeroCuantificadores,polaridadFrase = CalcularPolaridad(fraseProcesada)
            X_test.insert(i,[polaridadFrase,numeroPositivas,numeroNegativas,numeroCuantificadores])
    return np.asarray(X_train), np.asarray(X_test), Y_train, Y_test

# Main

## Analisis de sentimiento Enfoque: vector derivado de un analisis de sentimiento con lexicones

In [13]:
algoritmo = "SVM"
corpus = pd.read_csv('archivos/sentimientos.csv')
X_train, X_test, Y_train, Y_test = ClasificarPolaridadLexicon(corpus,algoritmo)

In [21]:
accuracyPromedio = 0
algoritmo="SVM"
numeroEjecuciones = 10000
for i in range(numeroEjecuciones):
    accuracy, kk = EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo)
    accuracyPromedio += accuracy
accuracyPromedio/numeroEjecuciones

0.7133105802048998